# <center>Ch7 GRPO详解 </center>

# 背景

### 1. 解决大模型微调的核心痛点
- **传统RLHF的局限性**：  
  - PPO等经典方法在大模型微调中存在 **高方差、训练不稳定** 的问题，尤其在生成长序列（如多步推理）时，稀疏奖励（仅最终结果打分）难以有效优化中间步骤。  
  - **GRPO的改进**：  
    - 通过 **分组（Group）策略** 和 **相对优势计算**，显著降低方差，提升策略更新的稳定性。  
    - 例如：在数学推理中，将不同推导路径分组，通过组间比较优化，避免无效探索。

### 2. 技术优势的显性化
- **细粒度策略优化**：  
  - GRPO通过 **组内协作+组间竞争** 的机制（类似多智能体强化学习），能更好处理多任务、多模态场景。  
  - 例如：在代码生成中，将不同功能的代码块分组优化，提升生成逻辑的连贯性。

- **计算效率提升**：  
  - 传统PPO需大量样本才能稳定训练，而GRPO通过分组减少冗余计算（如共享组内经验），更适合超大模型微调。

- **对齐（Alignment）需求的爆发**：  
  - 随着ChatGPT等大模型落地，**安全性和可控性**成为焦点。GRPO的分组设计可灵活整合人类偏好（如将“安全响应”和“创造性响应”分组优化），平衡多目标冲突。


# 1.GRPO技术解析

## 什么是GRPO

GRPO (Group Relative Policy Optimization 群体相关策略优化 )是一种针对大型语言模型的强化学习微调方法，是对传统PPO算法的改进和扩展。

### 核心架构组成
- **策略模型**：被训练的语言模型，初始通常是SFT模型
- **参考模型**：固定的初始模型，用于计算KL散度约束
- **奖励模型**：评估生成文本质量的模型，提供奖励信号
- **组计算模块**：核心创新部分，进行多样本生成和优势计算


GRPO的核心特点包括：

1. **基于组计算的优势估计**：对每个输入问题生成多个候选回答，通过组内比较计算优势函数
2. **双层KL控制机制**：通过微观和宏观两个层面的KL散度约束，保持模型稳定性
3. **灵活的监督方式**：支持结果监督(Outcome Supervision)和过程监督(Process Supervision)两种模式

## GRPO的主要优势

### 1. 更精细的奖励指导

- **差异化强化**：能够根据回答质量的细微差别提供相应强度的训练信号
- **惩罚机制**：不仅强化好的回答，还能惩罚质量差的回答
- **奖励区分度**：能够区分不同程度的"正确"回答，而不是简单的二分法

### 2. 训练稳定性提升

- **组内标准化**：通过组计算减少奖励估计的方差
- **双层KL控制**：避免策略过度偏离初始模型，保持生成文本的语言质量
- **无需Value模型**：简化了训练架构，减少了训练不稳定因素

### 3. 样本效率提高

- **多样本学习**：从每个问题生成的多个回答中同时学习
- **更丰富的比较信号**：提供更多对比样本，增强学习信号
- **标准化奖励**：减少不同任务难度带来的奖励尺度差异

### 4. 灵活的应用适配

- **支持不同监督粒度**：可以根据任务特点选择整体评估或逐步评估
- **可调整的参数**：G值、KL系数等参数可以根据任务需求调整
- **迭代训练支持**：支持多轮迭代强化，持续提升性能


## 适用场景

### 1. 需要高质量回答的应用

- **客服机器人**：需要提供准确、有帮助且友好的回答
- **内容创作助手**：需要生成高质量、符合要求的文本内容
- **专业领域问答**：如法律、医疗、技术支持等需要专业准确回答的场景

### 2. 多样化输出评估场景

- **开放式生成任务**：故事创作、文案生成等有多种可能正确答案的任务
- **个性化反馈系统**：需要根据用户偏好调整输出质量的应用
- **创意输出场景**：需要评估创意质量而非简单正确性的应用

### 3. 需要精细控制的应用

- **安全关键应用**：需要严格控制输出内容安全性的场景
- **教育辅助工具**：需要根据学习者水平提供适当难度解答的应用
- **专业写作辅助**：需要符合特定风格和标准的专业写作场景

### 4. 计算资源受限场景

- **不使用Value网络**：相比需要额外价值网络的方法更节省资源
- **可调整组大小**：可以根据可用计算资源调整G值大小
- **灵活的训练方案**：可以根据资源情况选择适当的监督方式

## 总结

GRPO是一种灵活、高效的强化学习微调方法，通过差异化梯度系数和组计算机制提升了大语言模型的性能。它特别适合需要高质量、多样化输出的应用场景，能够提供比传统方法更精细的模型行为调整能力。其无需Value网络的设计也使其在计算资源受限的情况下仍然可行，是当前大模型微调技术中的重要发展。


# 2.GRPO流程讲解

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250325225804336.png" width=100%></div>

## PPO架构（图上半部分）逐元素分析

### 1. 输入阶段（左侧）
- **输入q**：表示用户提问或指令
  - 格式：可能是自然语言文本、结构化查询或其他形式
  - 在大语言模型背景下通常是prompt或instruction
  - 这是整个生成过程的起点

### 2. Policy Model处理（核心组件）
- **策略模型**（图中为黄色标记，表示需要训练）
  - 内部结构：通常是Transformer架构的语言模型
  - 输入处理：接收q并进行token编码和上下文理解
  - 自回归生成：逐token生成输出序列o
  - 参数更新：通过优势A指导的策略梯度进行更新
  - 决策过程：基于当前上下文计算下一个token的概率分布

### 3. 输出处理（三路评估）
生成的输出o被发送到三个不同模块进行评估：

#### 3.1 Reference Model路径
- **参考模型**（图中蓝色，表示参数冻结）
  - 本质：通常是SFT（监督微调）后的基础模型
  - 功能：计算当前策略与参考行为的偏离程度
  - 具体计算：KL散度 $D_{KL}(\pi_\theta(·|q) || \pi_{ref}(·|q))$
  - 约束作用：防止策略过度偏离原始能力分布

#### 3.2 Reward Model路径
- **奖励模型**（图中蓝色，表示参数冻结）
  - 结构：通常是经过人类偏好训练的判别模型
  - 输入：策略模型生成的输出o及原始查询q
  - 输出：标量奖励值r，评估输出质量
  - 评估角度：可能包括有用性、安全性、真实性等维度
  - 结果用途：提供强化学习的主要训练信号

#### 3.3 Value Model路径
- **价值模型**（图中黄色，表示需要训练）
  - 结构：通常与策略模型类似但输出不同
  - 功能：评估当前状态的预期累积奖励
  - 输入：与策略模型相同的q和o
  - 输出：状态价值估计v
  - 结果用途：为优势计算提供基线估计

### 4. GAE（广义优势估计）计算
- **优势估计模块**
  - 输入：奖励值r和状态价值v
  - 功能：综合即时奖励和长期价值估计
  - 输出：优势值A，用于策略更新

### 5. 策略更新机制
- **利用优势信号**
  - PPO具体目标：$L(\theta) = E_t[\min(r_t(\theta)A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon)A_t)]$
  - 其中r_t(θ)为新旧策略概率比值：$r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{old}}(a_t|s_t)}$
  - 最终更新考虑三方面：优势信号、裁剪目标和KL约束

## GRPO架构（图下半部分）逐元素分析

### 1. 输入阶段（左侧）
- **输入q**：与PPO相同的用户查询或指令
  - 处理方式相同：token化并输入到模型
  - 在GRPO中，同一输入将产生多个不同输出


### 2. Policy Model多样化生成（核心创新）
- **策略模型**（图中黄色，需要训练）
  - 关键区别：生成G个不同输出{o₁, o₂, ..., oG}
  - 生成方式：可能采用以下策略
    - 不同温度参数（如0.7, 1.0, 1.3等）
    - 不同采样策略（top-k、随机种子等）
    - 相同参数多次采样

### 3. 并行评估路径

#### 3.1 Reference Model路径
- **参考模型**（图中蓝色，参数冻结）
  - 处理方式：每个输出oᵢ都与参考模型比较
  - 计算G组KL散度值
  - 每个输出可能有不同的KL预算
  - 结果用于约束训练，确保策略不偏离过远

#### 3.2 Reward Model路径
- **奖励模型**（图中蓝色，参数冻结）
  - 处理方式：评估每个输出oᵢ
  - 得到G组奖励值：{r₁, r₂, ..., rG}
  - 评估可以是：
    - 整体评估（结果监督）
    - 逐步评估（过程监督）
  - 整体评估：每个输出一个总体分数
  - 逐步评估：输出的每个步骤都有对应奖励

### 4. Group Computation（GRPO核心机制）
- **组计算模块**（相当于替代PPO中的Value Model和GAE）
  - 功能：通过组内比较计算每个输出的优势
  - 处理步骤：
    1. 接收所有G组奖励
    2. 进行组内标准化：$\hat{r}_i = \frac{r_i - \text{mean}(r)}{\text{std}(r)}$
    3. 根据监督类型计算优势

  - **结果监督下的优势计算**：
    - 简单直接，无需逐步评估

  - **过程监督下的优势计算**：
    - 每个token的优势是后续步骤奖励总和
    - 这为每个决策点提供更精准的信号

### 5. 多组优势更新机制
- **并行优势信号**
  - 得到G组优势值：{A₁, A₂, ..., AG}
  - 每组优势对应一个完整输出
  - 策略更新考虑所有优势信号
  - KL约束同样应用，防止策略偏离过远

## 两种架构的微观差异

### 1. 模型训练组成
- **PPO**：
  - 训练Policy Model和Value Model两个网络
  - 两个模型更新目标不同
  - 额外的参数和计算开销
  
- **GRPO**：
  - 仅训练单一Policy Model
  - 简化了训练复杂度
  - 减少了总参数量和内存需求

### 2. 信息流结构
- **PPO**：
  - 单输出多路径评估
  - 需要Value Model前向+反向传播
  - 顺序计算依赖性强
  
- **GRPO**：
  - 多输出并行评估
  - 组内比较替代Value估计
  - 更适合并行处理

### 3. 采样效率对比
- **PPO**：
  - 单次迭代生成单个样本
  - 需要多次迭代建立足够样本库
  - 样本之间可能上下文不同
  
- **GRPO**：
  - 单次迭代生成G个样本
  - 样本密度增加G倍
  - 所有样本共享相同上下文，便于比较


### 4. 奖励标准化机制
- **PPO**：
  - 通常需要手动设计奖励缩放或标准化
  - Value估计和真实奖励可能尺度不匹配
  
- **GRPO**：
  - 自动组内标准化：$\hat{r}_i = \frac{r_i - \text{mean}(r)}{\text{std}(r)}$
  - 天然适应不同难度任务和奖励范围
  - 减少了超参数调整需求

## 总结对比

这张图展示了GRPO通过三个关键创新超越PPO：

1. **结构简化**：移除Value Model，减少训练复杂度
2. **并行采样**：单次生成多样本，提高采样效率
3. **组内比较**：使用内部标准化替代Value估计，提高稳定性

这些设计使GRPO在保持简单实现的同时，解决了PPO在大规模语言模型应用中的核心挑战：样本效率低、训练不稳定和计算成本高。

# 3.Group Computation机制详解 

## 基本原理

### 核心思想
Group Computation的核心思想是：**用同一问题的多个不同回答之间的比较来确定每个回答的相对价值**，而不依赖额外的Value网络。

### 理论基础
1. **基线减法原理**：在强化学习中，从奖励中减去一个基线可以减少方差，同时不改变期望梯度
2. **相对评估原理**：人类难以给出绝对评分，但容易进行相对比较
3. **标准化统计原理**：同组数据标准化可以消除不同问题难度带来的尺度差异

### 与传统方法的对比
- **传统PPO**：需要训练一个Value网络来估计状态价值作为基线
- **Group Computation**：使用同组其他样本的统计量作为动态基线

## 关键步骤

### 1. 多样化采样
首先，我们有：
- 一个用户问题q
- 策略模型生成的G个不同输出: {o₁, o₂, ..., oG}
- 奖励模型评估的G个奖励: {r₁, r₂, ..., rG}

```json
问题q: "解释量子计算的基本原理"

生成的3个输出(简化为G=3):
o₁: "量子计算利用量子比特代替传统比特..."
o₂: "量子计算是一种使用量子力学现象的计算形式..."
o₃: "量子计算基于量子力学原理，使用量子叠加和纠缠..."
```

**技术细节**：
- 使用不同随机种子
- 采用不同温度参数
- 应用不同的top-p或top-k值
- 确保生成的回答足够多样

### 2. 奖励评估
使用奖励模型对每个回答进行评分，得到G个奖励值：{r₁, r₂, ..., rG}
```json

假设G=3
r₁ = 8.5
r₂ = 7.2
r₃ = 9.1
```

**技术细节**：
- 奖励可以来自单一奖励模型
- 也可以是多个评估指标的组合
- 在过程监督中，还需要评估每个推理步骤

### 3. 组内标准化
将奖励转换为标准分数，消除不同问题难度的影响：

**数学公式**:

r̂ᵢ = (rᵢ - mean(r)) / std(r)

其中:
- mean(r)是组内所有奖励的平均值
- std(r)是组内奖励的标准差

```json
mean(r) = (8.5 + 7.2 + 9.1) / 3 = 8.27
std(r) = √(((8.5-8.27)² + (7.2-8.27)² + (9.1-8.27)²) / 3) ≈ 0.784

标准化奖励:
r̂₁ = (8.5 - 8.27) / 0.784 ≈ 0.29
r̂₂ = (7.2 - 8.27) / 0.784 ≈ -1.36
r̂₃ = (9.1 - 8.27) / 0.784 ≈ 1.06
```

这一步确保了奖励值在相同尺度上，平均值为0，标准差为1。

**标准化效果**：
- 平均奖励变为0
- 标准差变为1
- 高于平均的回答获得正值
- 低于平均的回答获得负值

## 核心参数

### 1. 组大小G
- **定义**：每个问题生成的不同回答数量
- **常用值**：G = 8 或 G = 16 或者 G=64
- **影响**：
  * G越大，统计估计越稳定，计算资源消耗越多
  * G越小，训练速度越快，（如G≤2）会导致不稳定

**理论依据**：
- 统计理论表明方差随样本量的增加而减少
- 存在明显的边际收益递减

### 2. 奖励粒度
- **结果监督**：每个回答一个总体奖励
- **过程监督**：每个推理步骤一个奖励
- **混合监督**：结合两种方式的奖励信号

## 技术价值

### 1. 无需Value模型
- **简化架构**：减少了需要训练的模型数量
- **降低复杂性**：避免Value模型训练不稳定的问题
- **减少错配**：消除了策略模型与Value模型之间可能的不一致

### 2. 训练稳定性
- **自适应基线**：基线自动调整适应不同难度的问题
- **降低方差**：组内标准化减少了单个样本的噪声影响
- **鲁棒性**：对奖励尺度变化不敏感

### 3. 样本效率
- **多重学习信号**：每个问题产生G个学习样本
- **相对比较**：从同一问题的不同回答中提取更丰富的信息
- **零和学习**：好的回答被强化，差的回答被抑制

### 4. 泛化性能
- **相对评估**：关注同类问题中的相对表现
- **难度中和**：困难问题中的"相对好"回答仍获得正优势
- **领域适应性**：适用于各种文本生成任务

## 理解Group Computation的核心

要真正理解Group Computation的核心，需要从以下几个角度思考：

### 1. 统计视角
Group Computation本质上是一种**自举式基线估计法**，使用组内统计量作为奖励基线，而不是学习一个单独的Value函数。

标准化公式 r̂ᵢ = (rᵢ - mean(r)) / std(r) 的关键作用是：
- 消除了不同问题难度带来的奖励尺度差异
- 将奖励转换为相对性能度量
- 创造了一个自动调整的基线

### 2. 比较学习视角
Group Computation实现了**隐式的比较学习**：

- 高于平均的回答获得正优势，被强化
- 低于平均的回答获得负优势，被抑制
- 系统学习"什么是相对更好的回答"，而不只是"什么是好回答"


### 3. 算法设计视角
从算法设计角度，Group Computation是对传统Actor-Critic架构的巧妙替代：
- 传统方法：Actor + Critic(Value)网络
- Group Computation：Actor + 组内统计基线

这种设计保留了基线减法的方差减少优势，同时避免了Value估计的困难。

### 实际案例分析

考虑一个简单例子，假设我们有三个问题，每个问题生成G=4个回答：

**问题1**（简单问题）：
```json
回答奖励：{9.5, 9.0, 8.5, 8.0}
标准化后：{1.16, 0.39, -0.39, -1.16}
```

**问题2**（中等问题）：
```json
回答奖励：{7.0, 6.5, 6.0, 5.5}
标准化后：{1.16, 0.39, -0.39, -1.16}
```

**问题3**（困难问题）：
```json
回答奖励：{4.5, 4.0, 3.5, 3.0}
标准化后：{1.16, 0.39, -0.39, -1.16}
```

注意到：尽管原始奖励值差异很大，但标准化后的优势值完全相同！这正是Group Computation的强大之处 - 它自动适应不同难度的问题，确保模型学习的是"相对更好的策略"，而不受绝对奖励尺度的影响。

通过理解这些核心概念和案例，您应该能够更深入地把握Group Computation的本质 - 它不仅仅是一个技术细节，而是GRPO算法的核心创新，是使其在没有Value网络的情况下仍能高效稳定工作的关键机制。

Group Computation本质上是一种基于组内比较的优势估计方法，无需额外的Value Model。它有两个主要版本：结果监督(Outcome Supervision)和过程监督(Process Supervision)。



# 3.结果监督与过程监督

## 一、结果监督（Outcome Supervision）

### 基本原理
结果监督就像是**只关注最终成绩**的老师。无论你解题过程如何，只看你最后的答案对不对、好不好。

### 工作方式
1. 模型针对一个问题生成多个不同回答
2. 每个完整回答获得一个总体分数
3. 这个回答中的所有单词（tokens）都获得相同的"奖励"
4. 模型学习整体提高得分高的回答的生成概率，降低得分低的生成概率

### 具体流程
1. 对问题q生成G个不同回答{o₁, o₂, ⋯, oG}
2. 奖励模型评估每个回答，得到奖励值{r₁, r₂, ⋯, rG}
3. 标准化奖励：r̂ᵢ = (rᵢ - mean(r))/std(r)
4. 同一回答中的所有token共享相同的优势值：Âᵢₜ = r̂ᵢ

### 实例说明
假设问题："解释牛顿第二定律"

生成3个回答(G=3)：
```json
o₁: "牛顿第二定律描述了力与加速度的关系，即F=ma。这意味着物体的加速度正比于所受的力，反比于质量。"
o₂: "牛顿第二定律是运动定律之一，说明物体受力会产生加速度。"
o₃: "牛顿第二定律指出，物体的加速度与所受的合外力成正比，与质量成反比，即F=ma，其中F是力，m是质量，a是加速度。这一定律为经典力学提供了基础。"
```

奖励评分：

r₁ = 7.5    
r₂ = 5.0    
r₃ = 9.0    


标准化计算：

mean(r) = (7.5 + 5.0 + 9.0)/3 = 7.17    
std(r) = 1.65 (标准差计算)    

r̂₁ = (7.5 - 7.17)/1.65 = 0.20    
r̂₂ = (5.0 - 7.17)/1.65 = -1.32    
r̂₃ = (9.0 - 7.17)/1.65 = 1.11    


优势分配：
- o₁中的每个token都获得相同的优势值0.20    
- o₂中的每个token都获得相同的优势值-1.32
- o₃中的每个token都获得相同的优势值1.11

## 二、过程监督（Process Supervision）

### 基本原理
过程监督就像是**关注解题每一步**的老师。不仅看最终答案，还会检查你的每个推理步骤并分别打分。

### 工作方式
1. 模型针对一个问题生成多个不同回答
2. 将每个回答分解为多个推理步骤
3. 每个步骤单独获得一个分数
4. 每个单词（token）获得的"奖励"取决于它影响了哪些后续步骤
5. 早期单词影响更多步骤，因此获得的总奖励可能更多

### 具体流程
1. 对问题q生成G个不同回答{o₁, o₂, ⋯, oG}
2. 识别每个回答中的推理步骤，每步结束于index(j)
3. 奖励模型评估每个步骤，得到步骤奖励
4. 标准化每个步骤奖励：r̂ᵢ^index(j) = (rᵢ^index(j) - mean(R))/std(R)
5. 计算每个token的优势值为从该token开始的后续所有步骤的标准化奖励之和：Âᵢₜ = ∑(index(j)≥t) r̂ᵢ^index(j)


### 实例说明
继续上面的例子，但现在每个回答分为多个推理步骤：

回答o₁的步骤：
```json
步骤1(index=15): "牛顿第二定律描述了力与加速度的关系，"
步骤2(index=25): "即F=ma。"
步骤3(index=50): "这意味着物体的加速度正比于所受的力，反比于质量。"
```

步骤奖励：

r₁^15 = 6.0 (步骤1)    
r₁^25 = 8.0 (步骤2)   
r₁^50 = 7.0 (步骤3)   


同样为o₂和o₃计算步骤奖励(简化显示)：   

o₂: r₂^10 = 5.0, r₂^20 = 5.0    
o₃: r₃^20 = 8.0, r₃^35 = 9.5, r₃^65 = 9.0    

标准化所有步骤奖励(假设mean(R)=7.0, std(R)=1.5)：
```json
r̂₁^15 = (6.0-7.0)/1.5 = -0.67
r̂₁^25 = (8.0-7.0)/1.5 = 0.67
r̂₁^50 = (7.0-7.0)/1.5 = 0.0

r̂₂^10 = (5.0-7.0)/1.5 = -1.33
r̂₂^20 = (5.0-7.0)/1.5 = -1.33

r̂₃^20 = (8.0-7.0)/1.5 = 0.67
r̂₃^35 = (9.5-7.0)/1.5 = 1.67
r̂₃^65 = (9.0-7.0)/1.5 = 1.33
```

优势计算(选择性展示)：
- o₁中第1个token(t=1)的优势：
  Â₁,₁ = r̂₁^15 + r̂₁^25 + r̂₁^50 = -0.67 + 0.67 + 0.0 = 0.0
  
- o₁中第20个token(t=20)的优势：
  Â₁,₂₀ = r̂₁^25 + r̂₁^50 = 0.67 + 0.0 = 0.67
  
- o₁中第30个token(t=30)的优势：
  Â₁,₃₀ = r̂₁^50 = 0.0

- o₃中第1个token(t=1)的优势：
  Â₃,₁ = r̂₃^20 + r̂₃^35 + r̂₃^65 = 0.67 + 1.67 + 1.33 = 3.67

## 四、实际影响的对比

### 学习效果差异

 1. 奖励粒度
- **结果监督**：只关注最终结果，整个序列共享一个奖励
- **过程监督**：关注每个推理步骤，细化到多个奖励点

 2. 优势计算
- **结果监督**：所有token获得相同的优势值
- **过程监督**：token的优势值取决于其在序列中的位置，早期token考虑后续所有步骤的奖励

 3. 适用场景
- **结果监督**：适合答案质量容易整体评估的任务(如一般问答)
- **过程监督**：适合需要正确推理步骤的复杂任务(如数学、逻辑推理)

 4. 实现复杂度
- **结果监督**：实现简单，计算开销小
- **过程监督**：需要识别推理步骤，实现复杂，计算开销大

### 简单类比

结果监督就像考试只看最终答案对错，过程监督则像老师批改时会给解题步骤打分：

- 如果你只看最终答案（结果监督）：学生可能通过猜测得到正确答案，或者一个小计算错误导致整个解答被判错
  
- 如果你评价每一步（过程监督）：即使最终答案错了，学生也能知道哪些步骤是正确的，哪一步出了问题

## 五、何时使用哪种监督方式？

### 适合结果监督的场景
- 简短的事实类问答
- 评价标准简单明确的任务
- 最终结果比过程更重要的情况
- 计算资源有限时（实现更简单）

### 适合过程监督的场景
- 数学问题解答
- 多步逻辑推理
- 程序编写
- 科学解释
- 复杂决策过程

在实际应用中，GRPO允许根据不同任务特点选择合适的监督方式，甚至可以针对不同类型的问题动态切换监督模式，以获得最佳训练效果。

# 4.GRPO与PPO公式详解与应用

## 公式一：PPO目标函数

$\mathcal{J}_{PPO}(\theta) = \mathbb{E}[q \sim P(Q), o \sim \pi_{\theta_{old}}(O|q)] \frac{1}{|o|} \sum_{t=1}^{|o|} \min \left[ \frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{old}}(o_t|q, o_{<t})} A_t, \text{clip}\left(\frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{old}}(o_t|q, o_{<t})}, 1-\epsilon, 1+\epsilon\right) A_t \right]$


### 整体作用
这是PPO算法的核心目标函数，用于指导策略模型的参数更新，平衡探索与利用，同时确保更新稳定性。

### 参数详解
- **$\mathcal{J}_{PPO}(\theta)$**: PPO的目标函数，我们要最大化这个值
- **$\mathbb{E}[q \sim P(Q), o \sim \pi_{\theta_{old}}(O|q)]$**: 期望值，表示在问题分布和旧策略下采样
- **$\frac{1}{|o|}$**: 对输出序列长度的归一化，确保不同长度的输出有可比性
- **$\sum_{t=1}^{|o|}$**: 对输出序列中每个token位置求和
- **$\frac{\pi_{\theta}(o_t|q, o_{<t})}{\pi_{\theta_{old}}(o_t|q, o_{<t})}$**: 重要性采样比率，新旧策略的概率比
- **$A_t$**: 在位置t的优势值，表示该动作相对于平均表现的好坏
- **$\text{clip}(\cdot, 1-\epsilon, 1+\epsilon)$**: 裁剪函数，限制重要性比率在[1-ε,1+ε]范围内
- **$\min[\cdot,\cdot]$**: 取两者中较小值，实现保守更新


### 滑雪游戏示例
想象一个AI学习玩滑雪游戏：
- **问题q**: 当前滑雪场景（雪道状态、障碍物位置等）
- **输出o**: AI选择的动作序列（左转、右转、减速等）
- **旧策略$\pi_{\theta_{old}}$**: AI当前的滑雪策略
- **新策略$\pi_{\theta}$**: 正在学习的更新后策略
- **优势$A_t$**: 某个动作相比平均表现带来的额外收益

假设在一个陡峭转弯处：
- 旧策略给"右转"动作的概率是0.3
- 新策略给"右转"动作的概率是0.6
- 优势值为+2（表明右转是好的选择）
- ε设为0.2

计算过程：
1. 重要性比率: 0.6/0.3 = 2.0
2. 裁剪后: min(2.0, 1.2) = 1.2
3. 乘以优势: 1.2 × 2 = 2.4
4. 这个值将贡献给目标函数

这种裁剪机制确保了AI不会因为一次成功的右转就极端地调整策略。


## 公式二：奖励计算

$r_\tau = r_\varphi(q, o_{\leq t}) - \beta \log \frac{\pi_\theta(o_t|q, o_{<t})}{\pi_{ref}(o_t|q, o_{<t})}$


### 整体作用
这个公式计算调整后的奖励，综合考虑了原始奖励和KL散度惩罚项，平衡了性能提升和与参考模型的偏离程度。

### 参数详解
- **$r_\tau$**: 调整后的奖励值
- **$r_\varphi(q, o_{\leq t})$**: 奖励模型对当前输出的评分
- **$\beta$**: KL惩罚系数，控制对偏离参考模型的惩罚强度
- **$\log \frac{\pi_\theta(o_t|q, o_{<t})}{\pi_{ref}(o_t|q, o_{<t})}$**: 当前策略与参考策略的KL散度

### 滑雪游戏示例
继续滑雪游戏场景：
- **奖励模型评分**: AI完成一个漂亮转弯得到+5分
- **当前策略**: 在某种情况下以0.7的概率选择"减速后转弯"
- **参考策略**: 在同样情况下以0.4的概率选择"减速后转弯"
- **β值**: 设为0.5

计算过程：
1. 原始奖励: +5
2. KL项: 0.5 × log(0.7/0.4) ≈ 0.5 × 0.56 ≈ 0.28
3. 调整后奖励: 5 - 0.28 = 4.72

这表明虽然该动作获得了高奖励，但由于偏离了参考模型的行为，奖励被轻微降低。这防止AI过度专注于某些高奖励动作而忽略参考模型的整体能力。


## 优化点：奖励调整机制

### 1. 主要优化点

**KL正则化奖励设计**：公式二引入了直接将KL散度整合到奖励信号中的机制，而不是作为独立的约束项。

### 2. 具体优化内容

- **即时反馈机制**：传统方法通常在整个优化目标中添加KL惩罚，而公式二将其整合到每个时间步的奖励中
- **动态平衡**：在生成过程中动态平衡性能提升和能力保留
- **减少延迟反馈**：不需要等到完整序列生成后才施加KL约束



### 3. 在GRPO目标函数中的体现

在公式三（GRPO目标函数）中，这一优化通过以下方式体现：

公式二的调整奖励 → 计算优势值Â → 用于GRPO目标函数中的$Â_{i,t}$项


具体来说，公式三中的优势值$\hat{A}_{i,t}$是基于公式二调整后的奖励计算得出的：

$\hat{A}_{i,t}$ = 标准化(通过公式二计算的奖励)

这建立了一个直接链接：公式二改进了奖励信号，这些改进的奖励进而影响GRPO的优势计算和最终的策略更新。

## 公式三：GRPO目标函数

$\mathcal{J}_{GRPO}(\theta) = \mathbb{E}[q \sim P(Q), \{o_i\}_{i=1}^G \sim \pi_{\theta_{old}}(O|q)] \\\frac{1}{G} \sum_{i=1}^G \frac{1}{|o_i|} \sum_{t=1}^{|o_i|} \left( \min\left[\frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})} \hat{A}_{i,t}, \text{clip}\left(\frac{\pi_\theta(o_{i,t}|q, o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q, o_{i,<t})}, 1-\epsilon, 1+\epsilon\right) \hat{A}_{i,t}\right] - \beta D_{KL} \left[ \pi_\theta||\pi_{ref} \right] \right)$

### 整体作用
GRPO的目标函数，扩展了PPO的思想，通过多组输出的平均来提高样本效率和训练稳定性，同时直接集成KL约束。

### 参数详解
- **$\mathcal{J}_{GRPO}(\theta)$**: GRPO的目标函数
- **$\{o_i\}_{i=1}^G$**: G组不同的输出样本
- **$\frac{1}{G}$**: 对G组样本取平均
- **$\frac{1}{|o_i|}$**: 对每个输出序列长度归一化
- **$\hat{A}_{i,t}$**: 通过Group Computation计算的优势值
- **$\beta D_{KL}$**: KL散度约束项及其系数
- 其他项与PPO目标函数类似

### 滑雪游戏示例
在滑雪游戏中：
- **问题q**: 同一个滑雪场景
- **G组输出**: AI同时尝试G=3种不同的滑行路线
  - 路线1: 激进路线（高速通过弯道）
  - 路线2: 保守路线（减速谨慎通过）
  - 路线3: 平衡路线（适度减速后通过）

假设Group Computation后的优势值：
- 路线1优势: -1.2（风险太高）
- 路线2优势: 0.3（过于保守）
- 路线3优势: 1.5（恰到好处）

GRPO会综合考虑这三条路线的表现，同时计算与参考模型的偏离程度，最终一次性更新策略。这比PPO单独评估每条路线更有效率，也更稳定。





## 公式四：KL散度估计

$\mathbb{D}_{KL} \left[ \pi_\theta||\pi_{ref} \right] = \frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})} - \log \frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})} - 1$

### 整体作用
这是一个无偏KL散度估计器，用于精确计算当前策略与参考策略之间的差异程度，确保优化过程中不会过度偏离参考行为。

### 参数详解
- **$\mathbb{D}_{KL} \left[ \pi_\theta||\pi_{ref} \right]$**: 策略与参考模型之间的KL散度
- **$\frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})}$**: 参考策略与当前策略的概率比
- **$\log \frac{\pi_{ref}(o_{i,t}|q, o_{i,<t})}{\pi_\theta(o_{i,t}|q, o_{i,<t})}$**: 概率比的对数
- **$-1$**: 确保估计是无偏的修正项

### 滑雪游戏示例
在某个特定转弯处：
- **参考模型概率**: 选择"中等力度右转"的概率是0.5
- **当前策略概率**: 选择同一动作的概率是0.3

计算KL散度：
1. 概率比: 0.5/0.3 = 1.67
2. 对数项: log(1.67) ≈ 0.51
3. KL散度: 1.67 - 0.51 - 1 = 0.16

这个KL值告诉我们当前策略在这个决策点偏离参考模型的程度。值越大，偏离越严重，会导致更强的约束惩罚。






## 公式四的优化点：KL散度估计器

### 1. 主要优化点

**无偏KL散度估计**：公式四引入了一个数学上无偏的KL散度估计器，提高了KL约束的精确性。

### 2. 具体优化内容

- **估计精度提升**：传统KL估计可能有偏差，特别是在样本数量有限时
- **保证正值**：设计确保KL散度估计始终为正，符合KL散度的数学性质
- **减少采样变异**：提供更稳定的KL散度估计，减少训练波动
- **计算效率**：避免了需要大量采样才能准确估计KL散度的问题

### 3. 在GRPO目标函数中的体现

在公式三（GRPO目标函数）中，这一优化通过以下方式体现：


公式四的KL估计 → 直接用于公式三中的$D_{KL}[π_θ||π_{ref}]$项


具体来说，公式三末尾的KL约束项$\beta D_{KL} \left[ \pi_\theta||\pi_{ref} \right]$正是使用公式四计算得出的：

$\beta D_{KL} \left[ \pi_\theta||\pi_{ref} \right]$ = β × (由公式四计算的KL散度)

这建立了另一个直接链接：公式四提供了更准确的KL散度估计，这个改进的估计直接用于GRPO目标函数的约束项。

## 两个优化的协同作用

公式二和公式四的优化点协同工作，形成了一个完整的KL控制系统：

1. **双层KL控制**：
   - 公式二：在奖励层面整合KL约束（微观控制）
   - 公式四+公式三：在目标函数层面整合KL约束（宏观控制）

2. **互补性**：
   - 公式二解决"何时"和"如何"控制KL散度
   - 公式四解决"多准确"地衡量KL散度

3. **共同目标**：
   - 都旨在确保模型在优化性能的同时不会过度偏离参考行为
   - 前者是过程控制，后者是结果验证

## 公式在整体GRPO框架中的位置与关系

1. **公式一**（PPO目标函数）是传统PPO的核心，GRPO在此基础上扩展
2. **公式二**（奖励计算）定义了如何结合原始奖励和KL约束
3. **公式三**（GRPO目标函数）是GRPO的核心，整合了多组输出的优势和KL约束
4. **公式四**（KL散度估计）提供了准确计算策略偏离度的工具

在训练流程中：
1. 首先使用策略模型生成G组输出
2. 对每组输出计算奖励（使用公式二）
3. 使用Group Computation计算标准化优势
4. 计算与参考模型的KL散度（使用公式四）
5. 最后使用GRPO目标函数（公式三）更新策略模型

这种设计使GRPO在保持PPO核心思想的同时，通过多组输出的统计特性提高了训练效率和稳定性，使其特别适合大型语言模型的训练。

# 5.双层KL控制机制

## 一、微观KL控制

### 作用范围
- 在**每一组**（G个输出）中
- 对**每个具体回答**的奖励计算过程中

### 具体公式
$r_t = r_\phi(q, o_{\leq t}) - \beta \log \frac{\pi_\theta(o_t|q,o_{<t})}{\pi_{ref}(o_t|q,o_{<t})}$ 


### 工作过程

1. 对一个问题生成G个回答
2. 对每个回答：
   - 计算原始奖励 $r_φ$
   - 计算与参考模型的KL散度
   - 用KL散度调整奖励值
3. 这些调整后的奖励用于计算优势值

### 例子
假设一组内有3个回答：

问题："描述天气"

回答1: "天气晴朗" 
- 原始奖励：7分
- KL散度小
- 调整后奖励：约6.8分

回答2: "哇！天气太棒了！！"
- 原始奖励：8分
- KL散度大
- 调整后奖励：约5分

回答3: "今天是晴天"
- 原始奖励：7分
- KL散度小
- 调整后奖励：约6.9分


## 二、宏观KL控制

### 作用范围
- 在**整个训练过程**中
- 对**策略模型整体**的行为分布

### 具体公式
$J_{GRPO}(\theta) = \mathbb{E}[...] - \beta D_{KL}[\pi_\theta||\pi_{ref}]$


### 工作过程

1. 收集多组训练数据
2. 评估整体策略分布的变化
3. 在目标函数中施加KL约束
4. 影响整体的参数更新方向

### 例子

训练过程中观察到的整体变化：   
第1轮训练：
- 多组数据的风格基本保持
- 宏观KL约束较小

第10轮训练：
- 发现回答开始普遍活泼化
- 增加宏观KL约束

第20轮训练：
- 注意到专业术语使用减少
- 进一步调整宏观KL约束

## 三、两个层次的区别

### 1. 时间尺度
- **微观控制**：即时生效，作用于每个回答
- **宏观控制**：长期生效，作用于整体训练

### 2. 控制粒度   
**微观控制**：
- 单个回答级别
- 具体奖励调整
- 即时反馈

- **宏观控制**：
- 整体策略级别
- 参数更新方向
- 长期约束

### 3. 协同效果

微观控制：
- 处理具体的偏离行为
- 提供及时的纠正信号
- 影响单次学习效果

宏观控制：
- 维持整体行为分布
- 防止长期能力退化
- 保证训练稳定性


## 四、形象类比

想象训练一个厨师：

### 微观控制

- 对每道菜的调味进行指导
- 及时纠正用料的偏差
- 针对具体操作给出建议

### 宏观控制

- 保持餐厅整体的烹饪风格
- 确保菜品体系的完整性
- 维持长期的品质标准


## 五、总结

1. **微观KL控制**：作用于每组中的每个具体回答，通过调整奖励实现即时控制
2. **宏观KL控制**：作用于整个训练过程，通过目标函数约束实现长期控制

这种双层设计确保了：
- 短期行为不会过度偏离（微观控制）
- 长期能力得到保持（宏观控制）
- 训练过程更加稳定（双重保障）


## GRPO双层KL限制在实际训练中的应用

## 一、训练数据情况

问题a：生成10个答案 {a1, a2, ..., a10}   
问题b：生成10个答案 {b1, b2, ..., b10}   
问题c：生成10个答案 {c1, c2, ..., c10}   


## 二、微观KL控制（奖励计算阶段）

### 1. 对问题a的处理

对每个答案计算调整后的奖励：   
r_a1 = r_φ(a1) - β * KL(当前策略生成a1的概率/参考策略生成a1的概率)   
r_a2 = r_φ(a2) - β * KL(当前策略生成a2的概率/参考策略生成a2的概率)  
...    
r_a10 = r_φ(a10) - β * KL(当前策略生成a10的概率/参考策略生成a10的概率)   

### 2. 对问题b的处理

同样方式计算调整后的奖励：     
r_b1 = r_φ(b1) - β * KL(...)    
r_b2 = r_φ(b2) - β * KL(...)   
...   
r_b10 = r_φ(b10) - β * KL(...)   

### 3. 对问题c的处理

r_c1 = r_φ(c1) - β * KL(...)   
r_c2 = r_φ(c2) - β * KL(...)   
...   
r_c10 = r_φ(c10) - β * KL(...)   

这些调整后的奖励会用于计算每个答案的优势值。


## 三、宏观KL控制（策略更新阶段）

在收集了所有问题的所有答案后，在更新策略时：
```json
目标函数 = E.....[min(ratio * A, clip(ratio) * A)] - β * D_KL(当前整体策略||参考策略)
```

这个KL散度是在整个策略分布层面计算的，考虑了所有问题(a,b,c)的所有答案。

## 四、两层KL控制的区别

### 微观KL控制
- 作用于每个单独答案的奖励计算
- 在组内(每个问题的10个答案)进行
- 影响单个答案的学习信号强度

### 宏观KL控制
- 作用于整体策略更新
- 考虑所有问题所有答案的整体分布
- 影响整个模型的参数更新方向

## 五、实际训练流程

1. 对问题a：
   - 生成10个答案
   - 计算每个答案的原始奖励
   - 应用微观KL控制调整奖励
   - 计算优势值

2. 对问题b：
   - 重复同样过程
   
3. 对问题c：
   - 重复同样过程

4. 策略更新时：
   - 收集所有答案和优势值
   - 计算整体策略的KL散度
   - 应用宏观KL控制
   - 更新模型参数

这样，双层KL控制在不同层面共同作用，既保证了单个答案的质量，又维持了整体策略的稳定性。


# 6.总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250326170346825.png" width=100%></div>